In [ ]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Reshape
from keras.utils import to_categorical
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers import Dropout, InputLayer, LSTM
from keras.layers import Bidirectional, BatchNormalization
from keras.layers import TimeDistributed
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt
from keras.layers import Conv1D, AveragePooling1D, Dropout, UpSampling1D, BatchNormalization, LeakyReLU
from keras.regularizers import l1_l2
from sklearn.metrics import f1_score
from collections import Counter
from IPython import embed

In [ ]:
csv_path = '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/Output_files/'

In [ ]:
x_train_sm = pd.read_csv(csv_path + 'x_train_sm.csv')
y_train_sm = pd.read_csv(csv_path + 'y_train_sm.csv')

In [ ]:
len(x_train_sm)

3221904

In [ ]:
y_train_sm[y_train_sm['label'] == 1]

,Unnamed: 0,label
0,1561637,1.0
1,1530095,1.0
4,1793611,1.0
26,230138,1.0
29,1684813,1.0
...,...,...
3221880,1396749,1.0
3221885,1408819,1.0
3221886,1835878,1.0
3221891,2267558,1.0


In [ ]:
x_train = pd.read_csv(csv_path + 'x_train.csv', index_col=0)
y_train = pd.read_csv(csv_path + 'y_train.csv', index_col=0)

In [ ]:
len(x_train)

1341565

One hot encoding

In [ ]:
y_train.head()

,label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
x_train.shape

(1341565, 6)

In [ ]:
x_train.head()

,X_acc,Y_acc,Z_acc,X_gyr,Y_gyr,Z_gyr
0,4.435275,8.196063,2.974488,0.014215,-0.039157,-0.016744
1,4.186920,8.344455,2.908057,0.005771,-0.004480,-0.003345
2,4.544637,8.408659,2.890000,0.007967,0.022412,0.001159
3,4.849308,8.411614,2.900692,0.027778,-0.010670,-0.014223
4,4.509190,8.118649,2.847298,0.021577,-0.045498,-0.021111


In [ ]:
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
encoder = encoder.fit(y_train)
y_train_onehot = encoder.transform(y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
len(y_train_onehot)

1341565

In [ ]:
y_train_onehot = pd.DataFrame(y_train_onehot, columns = ['o1','o2','o3','o4'])

In [ ]:
y_train_onehot.head()

,o1,o2,o3,o4
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0


## Standard scaler

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_train_scaled = pd.DataFrame(x_train_scaled, columns = ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])

In [ ]:
def plot_loss(history):
    plt.title('Loss')
    plt.plot(history.history['loss'],  label='train')
    plt.plot(history.history['val_loss'],  label='validation')
    plt.show()

    plt.title('Accuracy')
    plt.plot(history.history['accuracy'],  label='train')
    plt.plot(history.history['val_accuracy'],label='validation')
    plt.show()

In [ ]:
x_train_scaled.head()

,X_acc,Y_acc,Z_acc,X_gyr,Y_gyr,Z_gyr
0,0.663850,0.023644,-0.555192,0.002976,-0.040439,0.066048
1,0.620531,0.045293,-0.567106,0.000352,-0.018272,0.083457
2,0.682926,0.054660,-0.570344,0.001034,-0.001083,0.089309
3,0.736068,0.055091,-0.568427,0.007191,-0.022230,0.069324
4,0.676743,0.012350,-0.578002,0.005264,-0.044492,0.060374


In [ ]:
def define_windows(x_train, y_train, windowsize):
  window_x=[]
  window_y=[]
  for i in range(0, len(x_train)-windowsize, windowsize+1):
    window_x.append([x_train.values[i:i+windowsize]])
    window_y.append([y_train.values[i:i+windowsize]])
  window_x=np.array(window_x).reshape(-1,windowsize,6)
  window_y=np.array(window_y).reshape(-1,windowsize,4)

  return window_x, window_y

In [ ]:
x_window, y_window = define_windows(x_train_scaled, y_train_onehot, 40)
print(x_window.shape, y_window.shape)

(32721, 40, 6) (32721, 40, 4)


In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x_window, y_window, test_size=0.1)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(29448, 40, 6) (29448, 40, 4)
(3273, 40, 6) (3273, 40, 4)


In [ ]:
def create_1dCNN():
  model = Sequential()
  model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same', input_shape=(40,6)))

  model.add(AveragePooling1D(2))
  model.add(Dropout(0.2))

  model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))

  model.add(AveragePooling1D(2))
  model.add(Dropout(0.2))

  model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
  model.add(UpSampling1D(size=2))
  model.add(Dropout(0.2))

  model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
  model.add(UpSampling1D(size=2))
  model.add(Dropout(0.2))

  model.add(Conv1D(4, kernel_size=3, activation='softmax', padding='same'))
  model.compile(optimizer=Adam(learning_rate=0.001),  loss='categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
cnn_model = create_1dCNN()

In [ ]:
cnn_fit = cnn_model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_val, y_val))

Epoch 1/20
921/921 [==============================] - 10s 9ms/step - loss: 0.6780 - accuracy: 0.7610 - val_loss: 0.5403 - val_accuracy: 0.8026
Epoch 2/20
921/921 [==============================] - 10s 11ms/step - loss: 0.5268 - accuracy: 0.8006 - val_loss: 0.4499 - val_accuracy: 0.8245
Epoch 3/20
921/921 [==============================] - 9s 10ms/step - loss: 0.4821 - accuracy: 0.8174 - val_loss: 0.4154 - val_accuracy: 0.8408
Epoch 4/20
921/921 [==============================] - 9s 9ms/step - loss: 0.4591 - accuracy: 0.8250 - val_loss: 0.3986 - val_accuracy: 0.8469
Epoch 5/20
921/921 [==============================] - 10s 11ms/step - loss: 0.4449 - accuracy: 0.8311 - val_loss: 0.3860 - val_accuracy: 0.8547
Epoch 6/20
921/921 [==============================] - 9s 10ms/step - loss: 0.4368 - accuracy: 0.8334 - val_loss: 0.3926 - val_accuracy: 0.8542
Epoch 7/20
921/921 [==============================] - 11s 12ms/step - loss: 0.4267 - accuracy: 0.8368 - val_loss: 0.3807 - val_accuracy: 0.85

In [ ]:
def create_1dCNN_updated():
  model = Sequential()
  model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same', input_shape=(40,6)))
  model.add(BatchNormalization())
  model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(AveragePooling1D(2))
  model.add(Dropout(0.2))

  model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(AveragePooling1D(2))
  model.add(Dropout(0.2))

  model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv1D(128, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(UpSampling1D(size=2))
  model.add(Dropout(0.2))

  model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv1D(64, kernel_size=3, activation='relu', padding='same'))
  model.add(BatchNormalization())
  model.add(UpSampling1D(size=2))
  model.add(Dropout(0.2))

  model.add(Conv1D(4, kernel_size=3, padding='same'))
  model.add(LeakyReLU(alpha=0.1))
  model.add(Conv1D(4, kernel_size=3, padding='same', activation='softmax', activity_regularizer=l1_l2(l1=0.01, l2=0.01)))

  model.compile(optimizer=Adam(learning_rate=0.001),  loss='categorical_crossentropy', metrics = ['accuracy'])
  return model


In [ ]:
cnn_new_model = create_1dCNN_updated()

In [ ]:
cnn_fit_2 = cnn_new_model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_val, y_val))

Epoch 1/20
921/921 [==============================] - 37s 37ms/step - loss: 1.1621 - accuracy: 0.8176 - val_loss: 1.0453 - val_accuracy: 0.8851
Epoch 2/20
921/921 [==============================] - 33s 36ms/step - loss: 1.0380 - accuracy: 0.8824 - val_loss: 0.9942 - val_accuracy: 0.9080
Epoch 3/20
921/921 [==============================] - 35s 38ms/step - loss: 1.0057 - accuracy: 0.8990 - val_loss: 0.9775 - val_accuracy: 0.9179
Epoch 4/20
921/921 [==============================] - 34s 36ms/step - loss: 0.9904 - accuracy: 0.9071 - val_loss: 0.9645 - val_accuracy: 0.9237
Epoch 5/20
921/921 [==============================] - 33s 36ms/step - loss: 0.9784 - accuracy: 0.9137 - val_loss: 0.9663 - val_accuracy: 0.9220
Epoch 6/20
921/921 [==============================] - 33s 36ms/step - loss: 0.9693 - accuracy: 0.9175 - val_loss: 0.9528 - val_accuracy: 0.9293
Epoch 7/20
921/921 [==============================] - 36s 39ms/step - loss: 0.9613 - accuracy: 0.9225 - val_loss: 0.9520 - val_accuracy:

In [ ]:
cnn_new_model.evaluate(x_val, y_val)

103/103 [==============================] - 1s 9ms/step - loss: 0.9378 - accuracy: 0.9397


[0.9377759695053101, 0.9397494792938232]

In [ ]:
Y_pred = cnn_new_model.predict(x_val)

103/103 [==============================] - 1s 8ms/step


In [ ]:
Y_pred.shape

(3273, 40, 4)

In [ ]:
Y_pred = Y_pred.reshape(-1, 4)
Y_val = y_val.reshape(-1, 4)
print(Y_pred.shape,Y_val.shape)

(130920, 4) (130920, 4)


In [ ]:
y_pred_bool = np.argmax(Y_pred, axis = 1)
y_val_bool = np.argmax(Y_val, axis = 1)

In [ ]:
f1_score(y_pred_bool,y_val_bool, average='macro')

0.9038698637253739

In [ ]:
counter_pred = Counter(y_pred_bool)
counter_true = Counter(y_val_bool)
print(counter_pred,counter_true)

Counter({0: 102532, 3: 16873, 2: 7066, 1: 4449}) Counter({0: 99327, 3: 20197, 2: 7040, 1: 4356})


## Load test data

In [ ]:
subject9_X = pd.read_csv(csv_path + 'subject9_X.csv',index_col=0)

In [ ]:
subject9_X.head()

,X_acc,Y_acc,Z_acc,X_gyr,Y_gyr,Z_gyr
0,0.396845,8.710000,-4.476044,0.001111,-0.008889,0.004137
1,0.485234,8.725234,-4.520000,-0.000020,0.006667,0.007778
2,0.460536,8.709464,-4.480000,0.001815,0.004525,0.002981
3,0.463828,8.708437,-4.470000,-0.000466,0.005089,0.002867
4,0.454513,8.715487,-4.489026,0.002387,0.004444,0.002696


In [ ]:
len(subject9_X)

9498

In [ ]:
subject10_X = pd.read_csv(csv_path + 'subject10_X.csv',index_col=0)
subject11_X = pd.read_csv(csv_path + 'subject11_X.csv',index_col=0)
subject12_X = pd.read_csv(csv_path + 'subject12_X.csv',index_col=0)

In [ ]:
print(len(subject10_X),len(subject11_X),len(subject12_X))

12270 12940 11330


In [ ]:
subject9_X_scaled = scaler.fit_transform(subject9_X)
subject9_X_scaled = pd.DataFrame(subject9_X_scaled, columns = ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])

In [ ]:
subject9_X_scaled.head()

,X_acc,Y_acc,Z_acc,X_gyr,Y_gyr,Z_gyr
0,-0.042342,0.097511,-1.735753,-0.007443,0.015090,-0.094789
1,-0.029772,0.100547,-1.743737,-0.007876,0.026628,-0.090565
2,-0.033285,0.097405,-1.736472,-0.007173,0.025039,-0.096128
3,-0.032816,0.097200,-1.734655,-0.008047,0.025457,-0.096261
4,-0.034141,0.098605,-1.738111,-0.006954,0.024979,-0.096460


In [ ]:
subject10_X_scaled = scaler.fit_transform(subject10_X)
subject10_X_scaled = pd.DataFrame(subject10_X_scaled, columns = ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])
subject11_X_scaled = scaler.fit_transform(subject11_X)
subject11_X_scaled = pd.DataFrame(subject11_X_scaled, columns = ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])
subject12_X_scaled = scaler.fit_transform(subject12_X)
subject12_X_scaled = pd.DataFrame(subject12_X_scaled, columns = ['X_acc', 'Y_acc', 'Z_acc', 'X_gyr', 'Y_gyr', 'Z_gyr'])

In [ ]:
print(len(subject10_X_scaled),len(subject11_X_scaled),len(subject12_X_scaled))

12270 12940 11330


In [ ]:
def define_test_windows(df_x, windowsize):
    window_x = []
    for i in range(0, len(df_x) - windowsize + 1, windowsize):
        window_x.append([df_x.values[i:i + windowsize]])
    # Add remaining data to window_x
    if len(df_x) % windowsize != 0:
        window_x.append([df_x.values[-windowsize:]])
    window_x = np.array(window_x).reshape(-1, windowsize, 6)
    return window_x

In [ ]:
subject9_X_scaled.shape[0]

9498

In [ ]:
subject9_X_window = define_test_windows(subject9_X_scaled,40) 
subject9_X_window.shape

(238, 40, 6)

In [ ]:
subject10_X_window = define_test_windows(subject10_X_scaled,40) 
subject11_X_window = define_test_windows(subject11_X_scaled,40) 
subject12_X_window = define_test_windows(subject12_X_scaled,40) 

In [ ]:
subject9_X_predict = cnn_new_model.predict(subject9_X_window)

8/8 [==============================] - 0s 9ms/step


In [ ]:
subject9_X_predict.shape

(238, 40, 4)

In [ ]:
subject9_X_predict = subject9_X_predict.reshape(-1,4)
subject9_X_predict = subject9_X_predict[:subject9_X.shape[0]]
subject9_X_predict.shape

(9498, 4)

In [ ]:
subject9_X_predict = np.argmax(subject9_X_predict, axis = 1)
subject9_X_predict

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
counter_subject9_X_predict= Counter(subject9_X_predict)
counter_subject9_X_predict

Counter({0: 1526, 1: 7954, 2: 18})

In [ ]:
subject10_X_predict = cnn_new_model.predict(subject10_X_window)
subject11_X_predict = cnn_new_model.predict(subject11_X_window)
subject12_X_predict = cnn_new_model.predict(subject12_X_window)

9/9 [==============================] - 0s 9ms/step


In [ ]:
subject10_X_predict = subject10_X_predict.reshape(-1,4)
subject10_X_predict = subject10_X_predict[:subject10_X.shape[0]]

subject11_X_predict = subject11_X_predict.reshape(-1,4)
subject11_X_predict = subject11_X_predict[:subject11_X.shape[0]]

subject12_X_predict = subject12_X_predict.reshape(-1,4)
subject12_X_predict = subject12_X_predict[:subject12_X.shape[0]]

In [ ]:
subject10_X_predict = np.argmax(subject10_X_predict, axis = 1)
subject11_X_predict = np.argmax(subject11_X_predict, axis = 1)
subject12_X_predict = np.argmax(subject12_X_predict, axis = 1)

In [ ]:
print(len(subject9_X_predict),len(subject10_X_predict),len(subject11_X_predict),len(subject12_X_predict))

9498 12270 12940 11330


In [ ]:
counter_subject10_X_predict= Counter(subject10_X_predict)
counter_subject10_X_predict

Counter({0: 1616, 1: 10647, 3: 7})

In [ ]:
counter_subject11_X_predict= Counter(subject11_X_predict)
counter_subject11_X_predict

Counter({0: 1478, 2: 9, 1: 11453})

In [ ]:
counter_subject12_X_predict= Counter(subject12_X_predict)
counter_subject12_X_predict

Counter({0: 3260, 1: 8066, 2: 4})

In [ ]:
subject9_X_predict

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
subject9 = pd.DataFrame(subject9_X_predict)

In [ ]:
subject9.shape

(9498, 1)

In [ ]:
subject9.to_csv('subject_009_01__y.csv',header=None, index = False)

In [ ]:
subject10 = pd.DataFrame(subject10_X_predict)
subject10.to_csv('subject_010_01__y.csv',header=None, index = False)
subject11 = pd.DataFrame(subject11_X_predict)
subject11.to_csv('subject_011_01__y.csv',header=None, index = False)
subject12 = pd.DataFrame(subject12_X_predict)
subject12.to_csv('subject_012_01__y.csv',header=None, index = False)